In [1]:
# # Mount the Google drive onto Colab Virtual Environment
# from google.colab import drive
# drive.mount('/content/drive')
root = '.'
# root = '/content/drive/My Drive/English Dataset'

In [2]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Embedding, Dense, CuDNNLSTM, LSTM
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import pickle
import os
from datetime import datetime

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
with open(os.path.join(root, 'en-mar_cleaned.en')) as f:
    lines = f.readlines()
    
lines = pd.DataFrame(lines, columns=['Src'])
lines['Tar'] = None

with open(os.path.join(root, 'en-mar_cleaned.ma')) as f:
    lines['Tar'] = f.readlines()

In [4]:
lines.Src = lines.Src.apply(lambda x: x.strip())
lines.Tar = lines.Tar.apply(lambda x: x.strip())
lines.Tar = '<START> ' + lines.Tar + ' <END>'

In [5]:
lines.shape

(33725, 2)

In [6]:
lines = lines[:1000]

In [7]:
lines.sample(10)

,Src,Tar
606,tom is out,<START> टॉम आउट झाला आहे <END>
160,look out,<START> सांभाळ <END>
457,i eat here,<START> मी इथे जेवतो <END>
200,who fell,<START> कोण पडलं <END>
405,ask anyone,<START> कोणालाही विचार <END>
943,tom is dead,<START> टॉम मेला आहे <END>
714,he has wine,<START> त्याच्याकडे वाईन आहे <END>
991,we are here,<START> आम्ही इथे आहोत <END>
552,lets talk,<START> बोलू या <END>
935,this is mad,<START> हा वेडेपणा आहे <END>


In [8]:
all_src_words = set()
for line in lines.Src:
    all_src_words = all_src_words.union(set(list(line.split(' '))))
    
all_tar_words = set()
for line in lines.Tar:
    all_tar_words = all_tar_words.union(set(list(line.split(' '))))

In [9]:
src_dict = dict()

for index, word in enumerate(sorted(all_src_words)):
    src_dict[word] = len(src_dict)
    
tar_dict = dict()

for index, word in enumerate(sorted(all_tar_words)):
    tar_dict[word] = len(tar_dict)

In [10]:
src_rev_dict = dict()

for key, value in src_dict.items():
    src_rev_dict[value] = key
    
tar_rev_dict = dict()

for key, value in tar_dict.items():
    tar_rev_dict[value] = key

In [11]:
max_len_src = max([len(sentence.split(' ')) for sentence in lines.Src])
max_len_src

4

In [12]:
max_len_tar = max([len(sentence.split(' ')) for sentence in lines.Tar])
max_len_tar

6

In [13]:
num_encoder_tokens = len(all_src_words)
num_decoder_tokens = len(all_tar_words)
(num_encoder_tokens, num_decoder_tokens)

(378, 680)

In [14]:
lines = shuffle(lines)
lines.head(10)

,Src,Tar
900,she bit him,<START> त्यांनी त्याला चावलं <END>
263,i saw tom,<START> मी टॉमला बघितलं <END>
538,its yours,<START> तुमचं आहे <END>
824,im resting,<START> मी आराम करतोय <END>
500,im a liar,<START> मी खोटारडा आहे <END>
56,call me,<START> मला फोन करा <END>
553,look again,<START> परत बघ <END>
499,ill start,<START> मी सुरुवात करते <END>
132,i can go,<START> मी जाऊ शकतो <END>
83,im tom,<START> मी टॉम आहे <END>


In [15]:
def encode_source(batch_input, batch_size):
    encoder_input_data = np.zeros((batch_size, max_len_src), dtype='float32')
    
    for i, input_text in enumerate(batch_input):
        for t, word in enumerate(input_text.split(' ')):
            if word not in src_dict.keys():
                word = 'UNK'
                
            encoder_input_data[i, t] = src_dict[word]
            
    return encoder_input_data

def encode_target(batch_output, batch_size):
    decoder_input_data = np.zeros((batch_size, max_len_tar), dtype='float32')
    decoder_target_data = np.zeros((batch_size, max_len_tar, num_decoder_tokens), dtype='float32')
    
    for i, target_text in enumerate(batch_output):
        for t, word in enumerate(target_text.split(' ')):
            if word not in tar_dict.keys():
                word = 'UNK'
                
            if t < len(target_text.split())-1:
                decoder_input_data[i, t-1] = tar_dict[word]
                
            if t > 0:
                decoder_target_data[i, t-1, tar_dict[word]] = 1
                
    return decoder_input_data, decoder_target_data

In [16]:
def generate_batch(X, y, batch_size):
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = encode_source(X[j:j+batch_size], batch_size)
            decoder_input_data, decoder_target_data = encode_target(y[j:j+batch_size], batch_size)
            
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [17]:
X, y = lines.Src, lines.Tar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
X_train.shape, X_test.shape

((900,), (100,))

In [18]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32
epochs = 50

In [19]:
latent_dim = 128

In [20]:
encoder_inputs = Input(shape=(None, ), name='Inputs')
enc_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True, name='English_Embeddings2')(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True, name='Encoder_LSTM')
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [21]:
decoder_inputs = Input(shape=(None, ), name='Outputs')
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero=True, name='Marathi_Embeddings')
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='Decoder_LSTM')
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Dense_Eng_Mar' + str(datetime.today().date()))
decoder_outputs = decoder_dense(decoder_outputs)

In [22]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [23]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [24]:
model.load_weights(os.path.join(root, 'translator_en_hi.h5'), by_name=True)

In [24]:
filepath = os.path.join(root, "best_model_en_ma.hdf5")
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [25]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size, 
                    callbacks=callbacks_list)

Epoch 1/50
28/28 [==============================] - 5s 191ms/step - loss: 4.3471 - acc: 0.1145 - val_loss: 4.0179 - val_acc: 0.0846

Epoch 00001: acc improved from -inf to 0.11454, saving model to ./best_model_en_ma.hdf5


/usr/local/lib/python3.6/site-packages/keras/engine/topology.py:2364: UserWarning: Layer Decoder_LSTM was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder_LSTM/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'Encoder_LSTM/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Epoch 2/50
28/28 [==============================] - 3s 96ms/step - loss: 3.7180 - acc: 0.1121 - val_loss: 3.7996 - val_acc: 0.0570

Epoch 00002: acc did not improve
Epoch 3/50
28/28 [==============================] - 3s 100ms/step - loss: 3.4667 - acc: 0.1193 - val_loss: 3.6904 - val_acc: 0.1158

Epoch 00003: acc improved from 0.11454 to 0.11934, saving model to ./best_model_en_ma.hdf5
Epoch 4/50
28/28 [==============================] - 2s 88ms/step - loss: 3.2414 - acc: 0.1430 - val_loss: 3.5457 - val_acc: 0.1232

Epoch 00004: acc improved from 0.11934 to 0.14296, saving model to ./best_model_en_ma.hdf5
Epoch 5/50
28/28 [==============================] - 3s 94ms/step - loss: 3.0521 - acc: 0.1660 - val_loss: 3.3981 - val_acc: 0.1541

Epoch 00005: acc improved from 0.14296 to 0.16597, saving model to ./best_model_en_ma.hdf5
Epoch 6/50
28/28 [==============================] - 2s 83ms/step - loss: 2.8688 - acc: 0.1933 - val_loss: 3.0941 - val_acc: 0.1590

Epoch 00006: acc improved from 0.

28/28 [==============================] - 2s 80ms/step - loss: 0.1745 - acc: 0.7163 - val_loss: 1.7982 - val_acc: 0.5103

Epoch 00039: acc improved from 0.71620 to 0.71633, saving model to ./best_model_en_ma.hdf5
Epoch 40/50
28/28 [==============================] - 3s 103ms/step - loss: 0.1576 - acc: 0.7174 - val_loss: 1.6549 - val_acc: 0.5292

Epoch 00040: acc improved from 0.71633 to 0.71741, saving model to ./best_model_en_ma.hdf5
Epoch 41/50
28/28 [==============================] - 2s 88ms/step - loss: 0.1378 - acc: 0.7176 - val_loss: 2.0385 - val_acc: 0.5276

Epoch 00041: acc improved from 0.71741 to 0.71760, saving model to ./best_model_en_ma.hdf5
Epoch 42/50
28/28 [==============================] - 3s 105ms/step - loss: 0.1227 - acc: 0.7179 - val_loss: 1.6152 - val_acc: 0.5156

Epoch 00042: acc improved from 0.71760 to 0.71787, saving model to ./best_model_en_ma.hdf5
Epoch 43/50
28/28 [==============================] - 3s 97ms/step - loss: 0.1093 - acc: 0.7176 - val_loss: 1.7772 

In [26]:
model.save(os.path.join(root, 'translator_en_ma.h5'))

/usr/local/lib/python3.6/site-packages/keras/engine/topology.py:2364: UserWarning: Layer Decoder_LSTM was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'Encoder_LSTM/while/Exit_2:0' shape=(?, 128) dtype=float32>, <tf.Tensor 'Encoder_LSTM/while/Exit_3:0' shape=(?, 128) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [27]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim, ))
decoder_state_input_c = Input(shape=(latent_dim, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [28]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tar_dict['<START>']
    
    stop_condition = False
    decoded_sentence = ''
    
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tar_rev_dict[sampled_token_index]
        decoded_sentence += ' ' + sampled_char
        
        if (sampled_char == '<END>') or (len(decoded_sentence) > 50):
            stop_condition = True
            
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        
        states_value = [h, c]
    return decoded_sentence

In [29]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [30]:
for i in range(10):
    k+=1
    (input_seq, actual_output), _ = next(train_gen)
    decoded_sentence = decode_sequence(input_seq)
    print('Input English sentence:', X_train[k:k+1].values[0])
    print('Actual Marathi Translation:', y_train[k:k+1].values[0][7:-5])
    print('Predicted Marathi Translation:', decoded_sentence[:-5])

Input English sentence: i see them
Actual Marathi Translation:  मी त्यांना बघतो 
Predicted Marathi Translation:  बघू बघू दिलं दिलं करतो करतो करतो करतो करतो करतो
Input English sentence: i know
Actual Marathi Translation:  मला माहीत आहे 
Predicted Marathi Translation:  माझा तुमच्यावर होता होता होता होता होता होता होत
Input English sentence: i was naked
Actual Marathi Translation:  मी नग्न होते 
Predicted Marathi Translation:  माझा होते होते होते होते होते होते होते होते होते
Input English sentence: it burned
Actual Marathi Translation:  जळलं 
Predicted Marathi Translation:  जळला जळला जळला जळला जळला जळला जळला जळला जळला जळला
Input English sentence: he can read
Actual Marathi Translation:  त्याला वाचता येतं 
Predicted Marathi Translation:  बघू काम काम काम काम काम काम काम काम काम काम का
Input English sentence: get started
Actual Marathi Translation:  सुरू करा 
Predicted Marathi Translation:  सुरू सुरू सुरू सुरू सुरू सुरू सुरू सुरू सुरू सुरू
Input English sentence: is he tall
Actual Marathi T